# 전처리 2

In [4]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
pd.options.mode.chained_assignment = None  # 경고 숨기기

# 1~4. 기존과 동일 --------------------- </br>
    (1) Date 컬럼 형식 맞추기 - '실제 주식 데이터'와 비교 위해서 </br>
    (2) 텍스트 정제된 ppc_Title 컬럼 만들기</br>
    (3) ppc_Title 컬럼 중복 확인 및 제거</br>
    (4) 띄어쓰기 기준 명사만 추출하여 Tokens 컬럼 만들기

In [5]:
### 4. 띄어쓰기 후 각 단어 형태소 분석 -> 명사만 추출
kiwi = Kiwi(typos='basic')
stopwords = Stopwords()

def tokenize_text(text):
    # words = kiwi.space(text)  # (1) 띄어 쓰기 교정      ##교수님 거에 있어서 추가했어!!!!!!!
    words = text.split()  # (2) 띄어쓰기로 텍스트 나누기

    token_list = []
    for word in words:
        tokens = kiwi.tokenize(word)  # (3) 각 단어를 형태소 분석
        for token in tokens:
            if token.tag in ['NNG', 'NNP']:   # (4) 명사(NNG, NNP)만 추출
                token_list.append(token.form)
    
    # token_list = ' '.join(token_list) # (5) 모든 문장을 하나의 string으로 join    ##교수님 거에 있어서 추가했어!!!!!!! Token 컬럼에  [ ] 없애주는 거야!
    return token_list

In [6]:
# 데이터 불러오기
news = pd.read_csv('hybe_01.Preprocessing_FIN.csv')
news.head()

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭,하이브 걸그룹 데뷔 서바이벌 프로그램 론칭
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발,단체 활동 중단 하이브 직원 주식 매도 적발
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락,공개 정보 이용 하이브 직원 후회 막심 사태 이후 하이브 급등 카카오 하락
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM,때 부진 엑소 와 갈등 악재
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용,하이브 올해 주식 공정 거래 절반 공개 정보 이용


# <span style="color:pink"> ◤ 전처리 과정 추가 ---------------------

### 5. 고유명사 직접 추가하여 tokens 수정하기 

In [7]:
# 사용자 정의 사전에 추가할 단어 목록
N_words = ['(여자)아이들', '하이브', '인생네컷', '투바투', '위버스', '프로미스나인', '이복현', '엠카', '빅히트', '금양', 'SM', 'YG', 'BTS', '김남국', '퀸카', 'AI', 'K팝', '세븐틴', '엔터', '불공정', '미공개', '미공시', '아이더', '프로미스나인', '순매수', 'LG에너지솔루션', 'NH아문디', 'K팝', 'ETF', 'JYP', 'W컨셉', '조회수', '소상공인', '뿌리기업', 'AI', '흔들', '꿈', '위버스',
           '엔터주', 'ENT', '최애', '줄상향', '한국', 'DM', '경창', 'DB하이텍', '첸백시', '블랙핑크', '하이브IM', '성료', '한앤', '두동', 'DNA', '권영식', '대거', '성장주', '업비트', '에너빌리티', '시도', '알스퀘어', '후끈', 'TXT', '롱런', '진해', '두나무', 'NFT', 'ESG', 'DDP', '아미', 'KBS', '유해진', '박해준', '배재현', 'FESTA', '속도', '대관료', 'CEO', '김길영', '블핑', 'GD',
           '스트레이키즈', '데이드림', '빌리버스', '위버스콘', '나라셀라', '최선호주', 'JTBC', '네카오', '그라피티', '차전지', '인공지능', '저작권', '응원봉', '군백기', '보라', 'TEAM', '오픈런', 'SK', 'LG', '눕방', '피독', '차일드라이프', 'LA', '만실', 'FML', '역조공', '송치형', 'MC', '한한령', '앤팀', 'NAVER', 'K', 'KT', '시총', '빅스코', 'CJ', 'CGV', 'SKT', '시가총액', '콘진원',
            '에브리씽', '권도형', 'MV', '우가팸', '에코', 'NH', 'CD', 'J팝', '싱글', '안동환', '김프로', '청양고추', '황민현', '이차전지', '셀트리온', '카뱅', '토뱅', 'SBS', 'IT', '미드낫', '포토이즘', '안우형', '세스템', '아이키', '최수영', '컴프야', 'NEXT', '최정은', '뉴진스', '돌싱글즈', '도기욱', 'CFO', '폴드', '와이지', '가상화폐', '버추얼', '휴먼', '유해진', '김희선', '포토카드',
            '키스오브라이프', '이동채', '한기정', '미운오리', '시큰둥', '카카오엔터', '김현용', '한터', 'HOT', 'MNET', '피땀눈물', '록빗', '침착맨', '한소희', 'NEXT', 'IP', '업살', '에스엠', 'NCT', '정호연', '어도어', 'OTT', '홍김동전', '민희진', '롤링스톤지', '아이랜드', 'APPLE', 'IPHONE', '페이커', '제베원', '하이브IM', '네옴시티', '숏폼', '야놀자', 'PPL', '롤라팔루자', '똥머리', '하이랜더','SUV', '네이버웹툰', 'POSCO홀딩스', '김건희', '구래커플', '친환경', '컴투스홀딩스', '떼창', '싹쓸', '쌍끌', '윈터', '써니', '구원투수', '인플루언서', '뿔', '마스크팩', '별이되어라', '아스달', '공인증서', '쌀국수', '김영찬', '맥신', '손창욱', '미투젠', 'MBC', '마스크걸', 'SW']


# NNP는 고유명사, NNG는 일반명사
for word in N_words:
    kiwi.add_user_word(word, "NNP")


In [8]:
news['Tokens'] = news['ppc_Title'].apply(tokenize_text)

### 6. 동의어 하나로 매핑 tokens 수정

In [10]:
# 동의어 매핑
replace_map = {
    'ARMY': '아미',
    'TXT': '투모로우바이투게더',
    '투바투': '투모로우바이투게더',
    'BTS': '방탄소년단',
    '방탄': '방탄소년단',
    'FESTA': '페스타',
    '블핑': '블랙핑크',
    '스키즈': '스트레이키즈',
    'GD': '지드래곤',
    'AI': '인공지능',
    '토요타': '도요타',
    '시총': '시가총액',
    'MV': '뮤비',
    '넥스트': '알유넥스트',
    'NEXT': '알유넥스트',
    '최수영': '수영',
    '와이지': 'YG',
    '에스엠': 'SM',
    'HOT': '핫',
    '땀': '피땀눈물',
    '피프티': '피프티피프티',
    'APPLE': '애플',
    'IPHONE': '아이폰',
    'POSCO홀딩스': '포스코홀딩스',
    '키스': '키스오브라이프',
    '포카': '포토카드',
    '엔터': '엔터테인먼트',
    '제이팝': 'J팝'
}

# 동의어 매핑을 tokens에 적용하여 수정
news['Tokens'] = news['Tokens'].apply(lambda x: [replace_map.get(item, item) for item in x])

### 7. 불용어 리스트와 제거 함수 만들어서 tokens 수정

In [11]:
# 불용어 리스트 생성
stopwords_list = ['선', '차', '유', '속', '조', '위', '때', '와', '올해', '인', '서', '발', '어치', '배', '줄', '판', '개', '주', '집', '월', '일', '분기', '만주', '전날', '연', '전', '첫날', '이상', '르', '달', '당장', '수', '파고', '올', '단위', '조간', '이달', '앤', '리', '중인', '하루', '앞', '이제', '올', '수량', '죽', '곳', '은', '파', '급', '입', '앞']

# 불용어 제거 함수 
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords_list]

In [12]:
# 불용어 제거 함수 적용
news_stop = news.copy()
news_stop['Stop_tokens'] = news_stop['Tokens'].apply(remove_stopwords)
news_stop.head()

,Date,Title,ppc_Title,Tokens,Stop_tokens
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭,"[하이브, 걸그룹, 데뷔, 서바이벌, 프로그램, 론칭]","[하이브, 걸그룹, 데뷔, 서바이벌, 프로그램, 론칭]"
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발,"[방탄소년단, 단체, 활동, 중단, 하이브, 직원, 주식, 매도, 적발]","[방탄소년단, 단체, 활동, 중단, 하이브, 직원, 주식, 매도, 적발]"
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락,"[미공개, 정보, 이용, 하이브, 직원, 후회, 막심, 사태, 이후, 하이브, 급등...","[미공개, 정보, 이용, 하이브, 직원, 후회, 막심, 사태, 이후, 하이브, 급등..."
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM,"[때, 부진, 엑소, 와, 갈등, 악재, SM]","[부진, 엑소, 갈등, 악재, SM]"
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용,"[하이브, 올해, 주식, 공정, 거래, 절반, 미공개, 정보, 이용]","[하이브, 주식, 공정, 거래, 절반, 미공개, 정보, 이용]"


### 8. 각 행의 리스트를 문자열로 결합하고 컴마 제거


In [13]:
news_stop_copy = news_stop.copy()

In [14]:
# Tokens 열을 문자열로 변환
news_stop_copy['Stop_tokens'] = news_stop_copy['Stop_tokens'].astype(str)

In [15]:
# Tokens 열에서 리스트 기호와 쉼표 제거
news_stop_copy['Stop_tokens'] = news_stop_copy['Stop_tokens'].str.replace("[\[\]']", "", regex=True)
news_stop_copy['Stop_tokens'] = news_stop_copy['Stop_tokens'].str.replace(",", "", regex=False)
news_stop_copy

,Date,Title,ppc_Title,Tokens,Stop_tokens
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭,"[하이브, 걸그룹, 데뷔, 서바이벌, 프로그램, 론칭]",하이브 걸그룹 데뷔 서바이벌 프로그램 론칭
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발,"[방탄소년단, 단체, 활동, 중단, 하이브, 직원, 주식, 매도, 적발]",방탄소년단 단체 활동 중단 하이브 직원 주식 매도 적발
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락,"[미공개, 정보, 이용, 하이브, 직원, 후회, 막심, 사태, 이후, 하이브, 급등...",미공개 정보 이용 하이브 직원 후회 막심 사태 이후 하이브 급등 카카오 하락
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM,"[때, 부진, 엑소, 와, 갈등, 악재, SM]",부진 엑소 갈등 악재 SM
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용,"[하이브, 올해, 주식, 공정, 거래, 절반, 미공개, 정보, 이용]",하이브 주식 공정 거래 절반 미공개 정보 이용
...,...,...,...,...,...
9185,2024-05-27,"[MD포토] 이희준, '완벽한 핸섬가이'",이희준 완벽한 핸섬가이,"[이희주, 완벽, 핸섬가]",이희주 완벽 핸섬가
9186,2024-05-27,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론,뉴진스 앨범 첫날 만장 민희진 해임 가처분 이번주 결론,"[뉴진스, 앨범, 첫날, 만장, 민희진, 해임, 처분, 이번, 주, 결론]",뉴진스 앨범 만장 민희진 해임 처분 이번 결론
9187,2024-05-27,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]",민희진 공격 VS 아일릿 보호 뉴진스 표절 둔 고소전 어떻게 봐야할까,"[민희진, 공격, 아일릿, 보호, 뉴진스, 표절, 고소, 전]",민희진 공격 아일릿 보호 뉴진스 표절 고소
9188,2024-05-27,"[MD포토] 공승연, '정말 재밌어요~'",공승연 정말 재밌어요,[공승],공승


In [16]:
# CSV 파일로 저장
news_stop_copy.to_csv('hybe_03.Preprocessing_01.csv', index=False)

---

### 9. 결측치 확인 및 제거

In [17]:
import pandas as pd 
news_check = pd.read_csv('hybe_03.Preprocessing_01.csv')
news_check


,Date,Title,ppc_Title,Tokens,Stop_tokens
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭,"['하이브', '걸그룹', '데뷔', '서바이벌', '프로그램', '론칭']",하이브 걸그룹 데뷔 서바이벌 프로그램 론칭
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발,"['방탄소년단', '단체', '활동', '중단', '하이브', '직원', '주식',...",방탄소년단 단체 활동 중단 하이브 직원 주식 매도 적발
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락,"['미공개', '정보', '이용', '하이브', '직원', '후회', '막심', '...",미공개 정보 이용 하이브 직원 후회 막심 사태 이후 하이브 급등 카카오 하락
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM,"['때', '부진', '엑소', '와', '갈등', '악재', 'SM']",부진 엑소 갈등 악재 SM
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용,"['하이브', '올해', '주식', '공정', '거래', '절반', '미공개', '...",하이브 주식 공정 거래 절반 미공개 정보 이용
...,...,...,...,...,...
9185,2024-05-27,"[MD포토] 이희준, '완벽한 핸섬가이'",이희준 완벽한 핸섬가이,"['이희주', '완벽', '핸섬가']",이희주 완벽 핸섬가
9186,2024-05-27,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론,뉴진스 앨범 첫날 만장 민희진 해임 가처분 이번주 결론,"['뉴진스', '앨범', '첫날', '만장', '민희진', '해임', '처분', '...",뉴진스 앨범 만장 민희진 해임 처분 이번 결론
9187,2024-05-27,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]",민희진 공격 VS 아일릿 보호 뉴진스 표절 둔 고소전 어떻게 봐야할까,"['민희진', '공격', '아일릿', '보호', '뉴진스', '표절', '고소', ...",민희진 공격 아일릿 보호 뉴진스 표절 고소
9188,2024-05-27,"[MD포토] 공승연, '정말 재밌어요~'",공승연 정말 재밌어요,['공승'],공승


In [18]:
# Tokens 결측치 확인
print(len(news_check[news_check['Stop_tokens'].isna()]))
news_check[news_check['Stop_tokens'].isna()]

2


,Date,Title,ppc_Title,Tokens,Stop_tokens
138,2023-06-07,[주요 신문 사설](7일 조간),일 조간,"['일', '조간']",NaN
4965,2023-12-11,[오전 주요 국제금융뉴스] (11일),일,['일'],NaN


In [19]:
### 5. 결측치 제거
news_check = news_check.dropna(subset=['Stop_tokens'])
news_check[news_check['Stop_tokens'].isna()]

,Date,Title,ppc_Title,Tokens,Stop_tokens


In [20]:
# CSV 파일로 저장
news_check.to_csv('hybe_03.Preprocessing_FIN.csv', index=False)

---

In [22]:
# 잘 저장 됐는지 확인
import pandas as pd 
news_check2 = pd.read_csv('hybe_03.Preprocessing_FIN.csv')

In [23]:
news_check2[news_check2['Stop_tokens'].isna()]

,Date,Title,ppc_Title,Tokens,Stop_tokens
